# Converting the CMIP6 Data Set to the OpenVisus Visualization Format

This Jupyter notebook provides an example of how one might take a portion of the CMIP6 Data Set and convert it to the OpenVisus Visualization Format.

## Assumptions

1. You are running this notebook via [OSG's OSPool Notebooks service](https://notebook.ospool.osg-htc.org) as a user of ap40.uw.osg-htc.org.

2. You have run through the steps in [the setup notebook](cmip6_setup.ipynb).

3. You have selected the `openvisuspy` kernel for this notebook via the Jupyter interface.

## Ensure that job credentials are available

N.B. This step is necessary only to work around a bug or misconfiguration that has yet to be fixed. Normally, these credentials are handled by HTCondor automatically.

In [ ]:
!echo | condor_store_cred add-oauth -s scitokens -i -

## Obtain a token for writing data into OSDF

N.B. HTCondor does not currently make it easy to copy a job's output directory into an OSDF origin that is not co-located with the access point. For the moment, our job (defined below) directly invokes `pelican object put`. The access token must be obtained separately, and included as part of the job's inputs.

## Submit the job

Set `container_image` to the location of the `openvisuspy` container image that you created in [the setup notebook](cmip6_setup.ipynb).

Set `ds_object` to the NetCDF file that you wish to convert.

Set `pelican_loc` to the location in a Pelican federation, e.g., OSDF, where the OpenVisus index should be copied to.

In [ ]:
import htcondor
import pathlib


## Define the container image for the OpenVisus software stack.

container_image = "osdf://ospool/ap40/data/brian.aydemir/openvisuspy-20240812-1020.sif"


## Define the object to convert.

federation_prefix = "osdf://aws-opendata/us-west-2"
ds_object = "nex-gddp-cmip6/NEX-GDDP-CMIP6/ACCESS-CM2/historical/r1i1p1f1/hurs/hurs_day_ACCESS-CM2_historical_r1i1p1f1_gn_1950.nc"
destination_dir = "openvisus"


## Define where to store the OpenVisus output.

pelican_loc = "pelican://osg-htc.org/nsdf/nasa-ecco/cmip6-demo/20240812-01"
pelican_token = "pelican_token"  # see notes above


## Remove log files from previous runs.

for ext in [".log", ".out", ".err"]:
    pathlib.Path(f"convert_dataset{ext}").unlink(missing_ok=True)


## Submit the job.

job_description = htcondor.Submit(
    f"""
    container_image = {container_image}
    args = python3 convert_dataset.py {destination_dir} $BASENAME({ds_object}) {pelican_loc} {pelican_token}

    transfer_input_files = convert_dataset.py, {federation_prefix}/{ds_object}, {pelican_token}
    transfer_output_files = {destination_dir}

    ## Save the job log, and standard output and error.
    log = convert_dataset.log
    output = convert_dataset.out
    error = convert_dataset.err

    ## Specify resource requests and other requirements.
    request_cpus = 2
    request_memory = 4G
    request_disk = 4G
    """
)

submitted_job = htcondor.Schedd().submit(job_description)

## Wait for the job to complete

In [ ]:
import demo_support

demo_support.wait_for_job(f"convert_dataset.log")

## Visualize the dataset

### Import libraries

In [ ]:
import os
import pathlib

import matplotlib.pyplot as plt
import numpy as np
import openvisuspy as ov

### Load the dataset

In [ ]:
token = pathlib.Path(pelican_token).read_text()

db = ov.LoadDataset(f"{pelican_loc}/visus.idx?authz={token}")

In [ ]:
print("Dimensions:", db.getLogicBox())
print("Total Timesteps:", len(db.getTimesteps()))
print("Field:", db.getField().name)

In [ ]:
ds_basename = os.path.basename(ds_object)
ds_parameters = os.path.splitext(ds_basename)[0].split("_")

if ds_parameters[-1][0] == "v":
    year = int(ds_parameters[-2])
else:
    year = int(ds_parameters[-1])

quality = 0  # full resolution = 0, coarse = -4, coarser = -8
day = 300
timestep = year * 365 + day

data3D = db.db.read(time=timestep, quality=quality)
data = data3D[:,:]
H, W = data3D.shape

fig, axes = plt.subplots(1, 1, figsize=(10, 10 * H / W))
axes.set_xlim(0, W)
axes.set_ylim(0, H)
axp = axes.imshow(data, extent=[0, W, 0, H], aspect="auto", origin="lower", cmap="viridis")
plt.colorbar(axp, location="right")
plt.show()